In [1]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
from keras.layers import *
import pickle, threading
import linecache
from keras.callbacks import Callback
import re
from keras.callbacks import ModelCheckpoint
from collections import Counter, defaultdict
from keras.utils import *
import keras
import tensorflow as tf

epochs = 50
trainable=True

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.__version__


'2.2.0'

In [3]:
nb_sequence_length=50
nb_embedding_dims=300

In [4]:

sys.version_info[0]

3

In [5]:
class WeightsSaver(Callback):
    def __init__(self, model, N):
        self.model = model
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'german_lm_twitter_20m.h5'
            print("model saved %s" %self.batch)
            self.model.save_weights(name)
        self.batch += 1

In [6]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [7]:
def twitter_tokenizer(textline:'takes line as an input'):
    
    textline = re.sub('@[\\w_]+', 'USER_MENTION', textline) # replaces @username with 'usermention'
    textline = re.sub('\\|LBR\\|', '', textline) # to replace linebreak
    textline = re.sub('\\.\\.\\.+', '...', textline) #????????
    textline = re.sub('!!+', '!!', textline)#?????????
    textline = re.sub('\\?\\?+', '??', textline)
    words = re.compile('[\\U00010000-\\U0010ffff]|[\\w-]+|[^ \\w\\U00010000-\\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    return(words)

In [ ]:
# creating the first 1m sentences for use
def create_train_data(filename:'name of original file', n_lines:'no of lines to use for training from the twitter data',
                     new_file_name:'name of file to create'):
    new_file = open(new_file_name,'wb')
    n_lines_read=0
    with open(filename,'rb') as f:
        for line in f:
            if n_lines_read == n_lines:
                break
            line = line.decode()
            ids, text = line.split('\t')
            n_lines_read+=1
            text = twitter_tokenizer(text)
            to_write = ' '.join(text)
            to_write+='\n'
#             print(to_write)
            new_file.write(to_write.encode())
            
#             assert n_lines_read<=10
    f.close()
    
            

In [ ]:
create_train_data('/home/jindal/notebooks/twitter_data/twitter-2017_de', 20000000, 
                  '/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter_20m')

In [8]:
window_size =51

In [ ]:
# converts the sentences to a fixed size of 51
def generate_sequences(out_file, in_file ) -> None:
    output = open(out_file,'wb')
    with open(in_file,'rb') as f:
        for line in f:
            text = line.decode().split()
            for i in range(len(text)):
                if i+1 >= window_size:
                    temp = text[i-window_size+1:i+1]
                else:
                    temp = ['0' for i in range(window_size - (i +1))] + text[:i+1]
                string = ' '.join(temp)+' \n'
                output.write(string.encode())
    output.close()


In [ ]:
generate_sequences('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m',
                   '/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter_20m')

In [ ]:
# assert
with open('twitter_dataset_sequences_20m') as f:
    for line in f:
        assert len(line.split()) == 51

In [9]:
line = linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m', 19)

print(line)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 Altere Rauchen Fetisch Sex Session https :// 



In [10]:
line_number=0
# lock = threading.Lock()

In [11]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [12]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    max_words = 50
    idx = 0
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    output_word = words[-1]
    if output_word in word_vectors_ft:
        output_wv = word_vectors_ft[output_word]
    else:
        output_wv = ft.get_word_vector(output_word.lower())
    return features_ft, output_wv

In [13]:
def sequential_generator(filename, batch_size, total_lines:'total lines in the file'):
    
    file_length = total_lines
    shuffled_indexes = range(1, file_length + 1) # start from 1 because linecache is 1 indexed
    nb_sequence_length = 50
    nb_embedding_dims = 300
    
    index_position = 0
    
    batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims)) # levy goldberg: not needed
    batch_labels = np.zeros((batch_size, 2)) # 2 because of simple task
    while True:
        # print(len(features))
        output_labels = []
        
        for i in range(batch_size):
            line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            batch_features_ft[i], output_wv = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            
#             print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
#             batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            output_labels.append(output_wv)
            index_position += 1
    
            if index_position == file_length:
                # shuffle indexes again
                shuffled_indexes = range(1, file_length + 1)
#                 shuffle(shuffled_indexes)
                index_position = 0
                break
        yield ([batch_features_ft], np.array(output_labels))


In [14]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES= 3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [15]:

model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(300, activation = "softmax")(model_concatenated)
model = Model(model_input_embedding, model_output)
model.compile(loss='cosine_proximity', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 50, 200)      320800      input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 50, 200)      0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      120200      leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv1d_2 (

In [16]:
total_lines = file_len('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m')

In [17]:
samples_per_epoch = total_lines
epochs = 1
batch_size = 64*2
steps_per_epoch = samples_per_epoch // batch_size
# checkpoint = ModelCheckpoint('lm_twitter_20m.h5', 
#                              monitor='val_acc', 
#                              verbose = 1, 
#                              save_best_only = True, 
#                              save_weights_only = True)

In [ ]:
multi_gpu_model = keras.utils.multi_gpu_model(model, gpus=2, cpu_relocation=True )

In [ ]:
print(keras.__version__
     )

In [ ]:
multi_gpu_model.compile(loss='cosine_proximity', optimizer='nadam', metrics = ['accuracy'])

In [ ]:

try:
    model.fit_generator(
        sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m', batch_size, total_lines), 
        steps_per_epoch=steps_per_epoch, epochs=epochs,callbacks=[WeightsSaver(model, 20000)]
        )
except Exception as e:
    print(e)
    model.save('german_lm_twitter_20m.h5')

Epoch 1/1
  29501/1014705 [..............................] - ETA: 48:17:23 - loss: -0.3320 - acc: 0.0402